In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

In [ ]:
!unzip tiny-imagenet-200.zip

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Afficher quelques exemples d'images
def imshow(img):
    img = img / 2 + 0.5     # inverse the normalization
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader)
# Use next(dataiter) instead of dataiter.next()
images, labels = next(dataiter)
imshow(torchvision.utils.make_grid(images))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Transformation des images (redimensionnement, normalisation, etc.)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Chargement des datasets Tiny ImageNet
train_dir = '/content/tiny-imagenet-200/train'
val_dir = '/content/tiny-imagenet-200/val'

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

# DataLoader pour entraîner et valider
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Charger un modèle pré-entraîné (par exemple, ResNet-50)
model = models.resnet50(pretrained=True)

# Gel des poids des couches pré-entraînées (fine-tuning)
for param in model.parameters():
    param.requires_grad = False

# Modifier la dernière couche pour l'adapter à Tiny ImageNet (200 classes)
model.fc = nn.Linear(model.fc.in_features, 200)

# Définir la fonction de perte (loss function)
criterion = nn.CrossEntropyLoss()

# Utiliser un optimiseur pour la dernière couche seulement (fine-tuning)
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Entraîner le modèle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fonction d'entraînement
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backpropagation et optimisation
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    return epoch_loss, accuracy

# Entraîner pendant plusieurs epochs
for epoch in range(10):  # Ajustez le nombre d'epochs
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
    print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")
